# PLANNING GRAPH PART

In [1]:
#%%

%load_ext autoreload
%autoreload 2

import numpy as np
import numpy.random as rd
from scipy import stats
from planning_graph.planning_graph import PlanningGraph, NoOpAction
from pddlpy.pddl import Operator
from typing import Set, Tuple, List

In [2]:
from planning_graph.planning_graph import PlanningGraph
from planning_graph.planning_graph_planner import GraphPlanner


planning_graph = PlanningGraph('domain/dock-worker-robot-domain.pddl', 
                               'domain/dock-worker-robot-problem.pddl')

graph = planning_graph.create(max_num_of_levels=10)
goal = planning_graph.goal
graph_planner = GraphPlanner()
layered_plan = graph_planner.plan(graph, goal)
print(layered_plan)
for i in range(len(layered_plan._layered_plan)):
    print(i)
    for a in (layered_plan[i]._plan):
        if not isinstance(a, NoOpAction):
            print(a)

LayeredPlan. Levels=4
0
1
2
3


In [3]:
def generate_feature_vec(planning_graph, state, max_level, visual = False, title = None):
    # draw graph first
    graph = planning_graph.create_with_state(state, max_num_of_levels=max_level, visualize = visual)
    if title is not None and visual:
        graph.visualize_png(title)
    
    # get action schema and output list
    act_schema = np.array(list(planning_graph._planning_problem._domain_problem.operators())) # store names of total action schema
    n = len(act_schema)  # length of action schema
    feature_vec = np.zeros(n+2*n**2+3) # return feature vec, first n is linear, second 2*n**2 is pairwise, last 3 is additional feature
    act_layers = list(graph.act.values()) # list of layers generated, ith value is the list of actions connencting i-1 th states to ith states layer

    # extract linear feature
    #-------------------------------------
    # ith value indicate the num of occurance for ith action of act_schema in the entire graph 
    counter = np.zeros(n)
    for act_layer in act_layers:
        if act_layer is not None:
            for a in act_layer:
                 if not isinstance(a, NoOpAction):
                        counter[act_schema == a.operator_name] += 1
    feature_vec[0:n] = counter
    
    # extract pair-wise feature
    #-------------------------------------
    # each pair a1, a2 is stored in n + [2*(n*a1+a2), 2*(n*a1+a2)+1]
    # e.g. when a1 is 1, a2 is 3, n is 5, store in 5 + [2*(8),  2*(8)+1]
    def to_index(n, index_a1, index_a2, adder):
        """
        return corresponding index in the position of the feature vector
        adder is either 0 or 1
        index_a1, index_a2 refer to move index in act_schema
        """
        return n+2*(n*index_a1+index_a2)+adder
    
    
    def append_to_dict(a, pre, eff_pos):
        """
        add action a into the dicitonary pre and eff_pos
        """
        for p in a.precondition_pos:
            current = pre.get(p)
            if current is None:
                current = [a]
            else:
                current.append(a)
            pre[p] = current
            
        for p in a.effect_pos:
            current = eff_pos.get(p)
            if current is None:
                current = [a]
            else:
                current.append(a)
            eff_pos[p] = current
            
#         for p in a.effect_neg:
#             current = effect_neg.get(p)
#             if current is None:
#                 current = [name]
#             else:
#                 current.append(name)
#             effect_neg[p] = current
#         return pre, eff_pos, eff_neg
            return pre, eff_pos


    # define dictionary variables for comparison purpose
    pre = {}
    eff_pos = {}
#     eff_neg = {}
    
    # add pre and eff into the empty dictionary for the first layer
    for a in act_layers[1]:
        if not isinstance(a, NoOpAction):
            pre, eff_pos = append_to_dict(a, pre, eff_pos)
   
    # loop through second to last action layers
    for i in range(2,len(act_layers)): 
        act_layer = act_layers[i]
        
        # update fecture vec for the entire layer
        for a2 in act_layer:
            if not isinstance(a2, NoOpAction):
                # count for num of occurances, use set to avoid multiple countsc
                s1 = set() # feature 1 where eff a1 and pre a2 has intersections
                s2 = set() # feature 2 where pre a1 and eff a2 has intersections          
                for p in a2.precondition_pos:
                    current = eff_pos.get(p)
                    if current is not None:
                        for a1 in current:
                            s1.add(a1) 

                for p in a2.effect_pos:
                    current = pre.get(p)
                    if current is not None:
                        for a1 in current:
                            s2.add(a1)
                            
                # add index to feature_vec based on set generated:
                index_a2 = int(np.where(a2.operator_name == act_schema)[0])
                for a1 in s1:
                    # update feature 1 for pair (a1, a2)
                    index_a1 = int(np.where(a1.operator_name == act_schema)[0])
                    feature_vec[to_index(n, index_a1, index_a2,0)]+=1
      
                for a1 in s2:
                    # update feature 2 for pair (a1, a2)
                    index_a1 = int(np.where(a1.operator_name == act_schema)[0])
                    feature_vec[to_index(n, index_a1, index_a2,1)]+=1

        # update pre and eff_pos for the entire layer
        for a2 in act_layer:
            if not isinstance(a2, NoOpAction):
                pre, eff_pos = append_to_dict(a2, pre, eff_pos)
                
                 
    
    # add heuristic value, number of layers and number of unsatisfied goals
    goal = planning_graph.goal
    graph_planner = GraphPlanner()
    layered_plan = graph_planner.plan(graph, goal)
    total_len = len(feature_vec)
    # number of layers:
    feature_vec[total_len - 3] = len(act_layers)
    # heuristic value: (number of total plans in the layer)
    h_v = 0
    for i in range(len(layered_plan._layered_plan)):
        for a in (layered_plan[i]._plan):
            if not isinstance(a, NoOpAction):
                h_v += 1
    feature_vec[total_len - 2] = h_v
    # unsatisfied goal (2 ** (last layer total pos num - goal state pos num))
    last = graph.prop[len(graph.prop)-1]
    feature_vec[total_len - 1] = 2 ** (len(last) - len(goal))
    

    return feature_vec
            
        

In [4]:
def find_operator(action : str, op_list: List[Operator]):
    """
    find an operator from the planning graph's ground operator lists
    
    return: the action operator if found
    """
    name_list = action.replace('(', '').replace(')', '').split(' ')
    for op in op_list:
        if op.operator_name == name_list[0]:
            if list(op.variable_list.values()) == name_list[1:]: return op
    return None

def apply_operation(action: Operator, state : Set[Tuple]):
    """
    apply an action onto the input state
    
    return: the new state
    """
    new_state = state.copy()
    for eff in action.effect_pos:
        new_state.add(eff)
    for eff in action.effect_neg:
        new_state.remove(eff)
    return new_state

def read_plan(plan_file_path: str):
    """
    read all the lines from a plan file directory, remove the last line containing cost
    
    return: a list containing the ground truth plan with length equal to total cost
    """
    with open(plan_file_path, "r") as f:

        # Read the lines of the file into a list of strings
        lines = [line.strip() for line in f.readlines()]

    return lines[:-1]


In [5]:
def generate_training_data(domain_file_path, task_file_path, plan_file_path, problem_num : int, visual = False, max_level=10):
    """
    generate the feature vector matrix X together with a cost vector y
    
    Returns:
    X : array, shape (plan_length-1, n_features)
        The input feature vec of states from initial states all the way towards the second-last state
    y : array, shape (plan_length-1, 2)
        The input cost vector. If it's a 2D array, the second column is the probelm_num
    """
    
    # generate graph
    planning_graph = PlanningGraph(domain_file_path, task_file_path, visualize = visual)
    graph = planning_graph.create(max_num_of_levels = max_level)
    plan_actions = read_plan(plan_file_path)
    ground_operators = planning_graph._planning_problem._get_ground_operators()
    
    # define output matrixes
    X = []
    y = []
    
    # loop from the final plan
    current_state = planning_graph._planning_problem.initial_state
    current_cost = len(plan_actions)
    for i in range(0,len(plan_actions)-1):
        X.append(generate_feature_vec(planning_graph, current_state, max_level))
#         X.append(generate_feature_vec(planning_graph, current_state, max_level, visual = True, title = f"test_image{i}th layer.png"))
        y.append(current_cost)
        current_action = find_operator(plan_actions[i], ground_operators)
        print(i, plan_actions[i],current_action.operator_name)
        current_state = apply_operation(current_action, current_state)
        current_cost -=1
        
    y = np.c_[y, problem_num * np.ones(len(y))]
    return np.asarray(X), np.asarray(y)

In [6]:
X, y = generate_training_data('domain/blocks/domain.pddl', 'domain/blocks/blocks/blocks4/task01.pddl', 'domain/blocks/plans/blocks4-task01_1800.out', 5)

0 (unstack b4 b1) unstack
1 (put-down b4) put-down
2 (unstack b1 b2) unstack
3 (put-down b1) put-down
4 (pick-up b2) pick-up


In [7]:
print(X.shape, y.shape)
print(type(X), type(y))
print(X)
print(type(y[0,0]))

(5, 39) (5, 2)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
[[1.4000000e+01 1.4000000e+01 3.8000000e+01 2.9000000e+01 0.0000000e+00
  0.0000000e+00 1.0000000e+01 3.2000000e+01 2.8000000e+01 9.6000000e+01
  0.0000000e+00 1.8000000e+01 2.4000000e+01 1.0000000e+01 0.0000000e+00
  0.0000000e+00 1.1000000e+01 0.0000000e+00 5.5000000e+01 2.5000000e+01
  3.0000000e+00 2.5000000e+01 0.0000000e+00 2.2000000e+01 1.1000000e+01
  6.9000000e+01 3.3000000e+01 9.9000000e+01 1.1000000e+01 0.0000000e+00
  5.0000000e+00 6.4000000e+01 5.1000000e+01 1.9900000e+02 3.1000000e+01
  3.5000000e+01 7.0000000e+00 6.0000000e+00 8.3886080e+06]
 [1.2000000e+01 1.2000000e+01 3.4000000e+01 2.3000000e+01 0.0000000e+00
  0.0000000e+00 8.0000000e+00 2.8000000e+01 2.4000000e+01 8.6000000e+01
  0.0000000e+00 1.5000000e+01 2.2000000e+01 8.0000000e+00 0.0000000e+00
  0.0000000e+00 7.0000000e+00 0.0000000e+00 4.6000000e+01 2.1000000e+01
  3.0000000e+00 2.1000000e+01 0.0000000e+00 1.8000000e+01 1.1000000e+01
  5.8000000e+0

# SVM PART

In [8]:
import itertools
import numpy as np
import numpy.random as rd
from sklearn import svm, linear_model
from sklearn.model_selection import KFold
from scipy import stats
from svm import synthetic_data as sdata

In [9]:
np.random.seed(2)
# p_index, test_data, cost = sdata.create_synthetic_data(25, 7, 100, 5, 64, 23, 3)
p_index, test_data, cost = sdata.create_synthetic_data(5, 5, 100, 2, 10, 23, 3)
print(f"test_data has shape: {test_data.shape}\ncost has shape: {cost.shape}\n")
print("first vector in data set:\n",np.round(test_data[0]))
cost_display = cost[0:min(len(cost),7)]
print("\nfirst 7 element in cost vector:\n",cost_display)
print("the index for problems: ",p_index)

test_data has shape: (28, 55)
cost has shape: (28, 2)

first vector in data set:
 [23.  8. 17. 12. 20.  2.  3.  2.  1.  1.  0.  2.  1.  2.  1.  3.  3.  4.
  1.  2.  4.  1.  1.  1.  3.  3.  1.  2.  4.  3.  2.  2.  3.  3.  4.  2.
  4.  3.  1.  4.  2.  4.  2.  1.  4.  4.  1.  2.  2.  2.  2.  0.  1.  3.
  4.]

first 7 element in cost vector:
 [[ 8  0]
 [15  0]
 [ 8  1]
 [39  1]
 [58  1]
 [37  1]
 [34  1]]
the index for problems:  [ 0  2 11 18 20 28]


In [10]:
"""
implementation based on
https://gist.github.com/agramfort/2071994
"""

import itertools
import numpy as np

from sklearn import svm, linear_model
from sklearn.model_selection import KFold


def transform_pairwise(X, y):
    """
    Transforms data into pairs for convex relaxation of kendal rank correlation coef
    In this method, all pairs are choosen, except for those that have the same target value or equal cost
    Parameters
    ----------
    X : array, shape (n_samples, n_features)
        The input feature vec of states from of several problems
    y : array, shape (n_samples,) or (n_samples, 2)
        The input cost vector. If it's a 2D array, the second column represents
        the problem index
    Returns
    -------
    X_trans : array, shape (k, n_feaures)
        Paired difference of features of si - sj
    y_trans : array, shape (k,)
        Output rank labels of values {-1, +1}, 1 represent larger cost for si compared to sj
    """
    X_new = []
    y_new = []
    if y.ndim == 1:
        y = np.c_[y, np.ones(y.shape[0])]
    comb = itertools.combinations(range(X.shape[0]), 2)
    for k, (i, j) in enumerate(comb):
        if y[i, 0] == y[j, 0] or y[i, 1] != y[j, 1]:
            # skip if same target or different group
            continue
        X_new.append(X[i] - X[j])
        y_new.append(np.sign(y[i, 0] - y[j, 0])) # y = 1 if feature vec got a larger cost
        if y_new[-1] != (-1) ** k:
            y_new[-1] = - y_new[-1]
            X_new[-1] = - X_new[-1]
    return np.asarray(X_new), np.asarray(y_new)


class RankSVM(svm.LinearSVC):
    """
    Performs pairwise ranking svm with an underlying LinearSVC model
    initialise with a C of regularization term
    default using hinge loss
    """
    
    def __init__(self, C = 1.0):
        super(RankSVM, self).__init__()
        self.C = C
        self.loss = 'hinge'
        
        
    def fit(self, X, y):
        """
        Fit a pairwise ranking model.
        Parameters
        ----------
        X : array, shape (n_samples, n_features)
        y : array, shape (n_samples,) or (n_samples, 2)
        Returns
        -------
        self
        """
        X_trans, y_trans = transform_pairwise(X, y)
        super(RankSVM, self).fit(X_trans, y_trans)
        return self

    def predict(self, X):
        """
        Predict an ordering on X. For a list of n samples, this method
        returns a list from 0 to n-1 with the relative order of the rows of X.
        Parameters
        ----------
        X : array, shape (n_samples, n_features)
        Returns
        -------
        ord : array, shape (n_samples,)
            Returns a list of integers representing the relative order of
            the rows in X.
        """
        if hasattr(self, 'coef_'):
            return np.argsort(np.dot(X, self.coef_.T).flatten())
        else:
            raise ValueError("Must call fit() prior to predict()")

    def score(self, X, y):
        """
        Because we transformed into a pairwise problem, chance level is at 0.5
        """
        X_trans, y_trans = transform_pairwise(X, y)
        return np.mean(super(RankSVM, self).predict(X_trans) == y_trans)


In [11]:
my_svm = RankSVM()

In [12]:
my_svm.fit(X, y)
my_svm.predict(X)

D:\anaconda\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


array([4, 2, 3, 1, 0], dtype=int64)